<a href="https://colab.research.google.com/github/Aniketsudke/Aniketsudke/blob/main/Dubliacte_Question_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sqlalchemy psycopg2-binary pandas nltk sentence-transformers scikit-learn

In [ ]:
from google.colab import userdata
userdata.get('DATABASE_URL')
DATABASE_URL=(userdata.get('DATABASE_URL'))


In [ ]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(
    DATABASE_URL,
    connect_args={
        "sslmode": "require"
    }
)

In [ ]:
engine.dispose()

query = """
SELECT
  q.id,
  q.slug,
  q.title,
  q.content,
  q."subjectId",
  q."topicId",
  q."subtopicId",
  STRING_AGG(o.content, ' || ' ORDER BY o.id) AS options_text
FROM "Question" q
LEFT JOIN "Option" o ON o."questionId" = q.id
WHERE q.content IS NOT NULL
  AND q."isPublished" = true
GROUP BY q.id
"""
df = pd.read_sql(query, engine)

df.head()

,id,slug,title,content,subjectId,topicId,subtopicId,options_text
0,000d0266-defe-4146-9cac-406b9747c6dc,neet-foam-breaker-identification-in-a-bioreact...,Foam Breaker Identification in a Bioreactor,Identify the part of a bioreactor which is use...,70821584-88d7-4205-9f32-4dd5ecde20b9,725e60b9-c519-499b-bac7-33e23dee0e89,8e55ed3a-cb12-4666-8f7b-9df00bda8d53,D || A || C || B
1,001d387d-1cd8-4cad-80eb-421cd87f5fd6,neet-mole-fraction-in-a-100-m-aqueous-solution...,Mole Fraction in a 1.00 m Aqueous Solution (NE...,What is the mole fraction of the solute in a *...,959f2ce7-b37a-4325-a9d0-d46c2ba3ef59,3451965e-23bc-46f0-bc45-e94ae123a82e,e65d16e3-940c-4b3d-99f5-355400d05015,0.177 || 1.770 || 0.0354 || 0.0177
2,00250252-ca3d-49ba-8f20-a63c16fd68f5,system-of-linear-equations-with-infinitely-man...,System of linear equations with infinitely man...,Let the system of equations be given by the th...,c47c9c14-2dd7-42fc-823d-d3f550a0e471,8b7916e8-9ee7-41bd-a45c-ad5b85fa522d,98d8b3b4-184f-47e9-b029-40de5c27655e,6 || 4 || 3 || 5
3,00281d43-810e-4f74-922e-99bbf5bb1aeb,aipmt-2003-stone-whirled-in-vertical-circle,AIPMT 2003 - Stone Whirled in Vertical Circle,A stone is tied to a string of length $l$ and ...,fdee61f5-0236-44d2-b614-a1da45659ca6,ad95e7ff-85cf-4597-a3fa-04fdeeae6929,df97624f-fce8-43ef-92e5-ca853c1cd094,$u-\sqrt{u^{2}-2 g l}$ || $\sqrt{u^{2}-g l}$ |...
4,0032fe98-4c74-43e8-9151-c9c7860a5dea,herbicide-resistant-gm-crops-main-objective,Herbicide-resistant GM crops - Main objective,Main objective of production/use of herbicide ...,70821584-88d7-4205-9f32-4dd5ecde20b9,393a9cdf-c197-4d9e-b551-4c0339d3abd6,ff3a2656-7c6d-4b37-a2aa-756d165ff304,reduce herbicide accumulation in food articles...


In [ ]:
import re
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
STOP_WORDS = set(stopwords.words("english"))

def normalize(text):
    if not text:
        return ""
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    return " ".join(w for w in text.split() if w not in STOP_WORDS)

df["norm_content"] = df["content"].apply(normalize)
df["norm_title"] = df["title"].apply(normalize)
df["norm_options"] = df["options_text"].apply(normalize)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
df["content_emb"] = list(
    model.encode(df["norm_content"].tolist(), show_progress_bar=True)
)

df["options_emb"] = list(
    model.encode(df["norm_options"].tolist(), show_progress_bar=True)
)

df["title_emb"] = list(
    model.encode(df["norm_title"].tolist(), show_progress_bar=True)
)


Batches:   0%|          | 0/215 [00:00<?, ?it/s]

Batches:   0%|          | 0/215 [00:00<?, ?it/s]

Batches:   0%|          | 0/215 [00:00<?, ?it/s]

In [ ]:
ADMIN_BASE_URL = "https://admin.rankmarg.in/admin/duplicate-detector"

In [ ]:
CONTENT_W = 0.6
OPTIONS_W = 0.3
TITLE_W = 0.1

SIM_THRESHOLD = 0.82  # semantic threshold


duplicates = []

GROUP_KEYS = ["subjectId", "topicId", "subtopicId"]

for _, group in df.groupby(GROUP_KEYS):
    group = group.reset_index(drop=True)

    for i in range(len(group)):
        for j in range(i + 1, len(group)):

            content_sim = cosine_similarity(
                [group.loc[i, "content_emb"]],
                [group.loc[j, "content_emb"]]
            )[0][0]

            options_sim = cosine_similarity(
                [group.loc[i, "options_emb"]],
                [group.loc[j, "options_emb"]]
            )[0][0]

            title_sim = cosine_similarity(
                [group.loc[i, "title_emb"]],
                [group.loc[j, "title_emb"]]
            )[0][0]

            final_score = (
                CONTENT_W * content_sim +
                OPTIONS_W * options_sim +
                TITLE_W * title_sim
            )

            if final_score >= SIM_THRESHOLD:
                duplicates.append({
                    "similarity_score": round(final_score * 100, 2),
                    "link": f"{ADMIN_BASE_URL}?slug1={group.loc[i, 'slug']}&slug2={group.loc[j,'slug']}",
                })

dup_df = pd.DataFrame(duplicates)
dup_df.sort_values("similarity_score", ascending=False)


,q1_id,q2_id,similarity_score,subjectId,topicId,subtopicId,link_1,link_2,q1_content,q2_content
248,298c88b8-804f-4c74-a406-02dcb49a9c55,3eed9aab-ac6e-4521-9923-8b50cbf2fd8a,100.000000,c47c9c14-2dd7-42fc-823d-d3f550a0e471,432ace1e-933f-4771-8136-4bafdc859c77,a6327c6c-ad05-41d9-9ae6-426f53e12ce3,https://admin.rankmarg.in/admin/questions/grea...,https://admin.rankmarg.in/admin/questions/grea...,"If $[x]^2 - 5[x] + 6 = 0$, where $[.]$ denotes...","If $[x]^2 - 5[x] + 6 = 0$, where $[.]$ denotes..."
234,6b7e3ae2-57c9-4d5d-8aa4-2b8e745d80ef,6d0e14a8-ce87-4f6a-bb1f-ae7dce897916,100.000000,959f2ce7-b37a-4325-a9d0-d46c2ba3ef59,e04a4e98-9cb3-4fe9-94dc-4c1fa12ab587,a73a2a59-5125-4a99-8a97-834b0d0c9313,https://admin.rankmarg.in/admin/questions/stab...,https://admin.rankmarg.in/admin/questions/stab...,The stability of $+1$ oxidation state among $A...,The stability of $+1$ oxidation state among $A...
161,37b9a743-f7df-4fe3-93da-097268db25f1,e1e7674c-64f8-429e-9607-48e1070bf4d3,99.970001,959f2ce7-b37a-4325-a9d0-d46c2ba3ef59,4b162d87-5907-4399-ae9e-894d534c3d77,12992952-fbc4-4b7c-9e6f-fc1e25742b17,https://admin.rankmarg.in/admin/questions/unit...,https://admin.rankmarg.in/admin/questions/unit...,Unit of entropy is,Unit of entropy is
66,dd741aeb-35e5-4004-8398-11ecb1a3ec1a,f5d4067f-645e-45f5-bdd2-ca7c1332fbef,99.949997,70821584-88d7-4205-9f32-4dd5ecde20b9,7b7ba8b3-7fe8-4906-9d9e-850a1e16658b,2efd14d3-3b56-4295-aa59-556839702a29,https://admin.rankmarg.in/admin/questions/aipm...,https://admin.rankmarg.in/admin/questions/aipm...,The maximum growth rate occurs in :-,The maximum growth rate occurs in :-
274,d9c90c3f-6a65-4cdf-80aa-f0f92a21472b,eff6fb0b-0c84-4557-a000-5b863ae08d6a,99.940002,c47c9c14-2dd7-42fc-823d-d3f550a0e471,8ca95646-0f54-46b2-b29d-b40e51424d49,816a8409-e0d8-4e74-acc2-327f5412448a,https://admin.rankmarg.in/admin/questions/unit...,https://admin.rankmarg.in/admin/questions/unit...,Let $\vec a=\hat i+2\hat j+\hat k$ and $\vec b...,Let $\vec{a} = \hat{i} + 2\hat{j} + \hat{k}$ a...
...,...,...,...,...,...,...,...,...,...,...
363,2eb91ca7-f40d-4f13-b11d-c6c2e9d029ec,32f4c3b2-6d4b-4c98-a9d5-aef44b43c63a,82.220001,fdee61f5-0236-44d2-b614-a1da45659ca6,81c0aabd-c944-4962-afe0-2efe06d4b8fc,7f92d00e-5649-4244-8eea-78229c9ae703,https://admin.rankmarg.in/admin/questions/youn...,https://admin.rankmarg.in/admin/questions/youn...,In the Youngs double slit experiment $λ$ is th...,"In Young's double slit experiment, the intensi..."
1,7e68b72e-d468-444c-bdde-3142bc97014e,d415c23d-b4d2-4bc9-a9b0-1d9c73f8ad82,82.120003,70821584-88d7-4205-9f32-4dd5ecde20b9,22728998-08ba-4a24-b742-604ebd6faa1a,500d1519-c145-495c-9981-b843c54413af,https://admin.rankmarg.in/admin/questions/herb...,https://admin.rankmarg.in/admin/questions/herb...,Herbarium sheet provides the information of,The name of a plant written on herbarium sheet...
175,5f1aed73-2f0a-4ab1-9554-5f8b4cdb70ec,c449b9cb-8f07-42da-894e-8e3cf6b3d01a,82.110001,959f2ce7-b37a-4325-a9d0-d46c2ba3ef59,6e3888cd-dc94-4c54-b576-2c915ab81362,fab6ece0-1916-42e3-80c6-be80badfb271,https://admin.rankmarg.in/admin/questions/equi...,https://admin.rankmarg.in/admin/questions/equi...,The equilibrium constants of the following rea...,If the equilibrium constant for\n\n$$\mathrm{N...
297,287c382f-66fb-4649-8059-71b10210b86d,8be09c27-deca-47c9-8a3b-0d3155d3e8fe,82.070000,c47c9c14-2dd7-42fc-823d-d3f550a0e471,db44f5f2-895f-4a2b-9506-dbbc691e45f5,8ac1bf23-9882-498d-90c7-3a5d70e58d0d,https://admin.rankmarg.in/admin/questions/smal...,https://admin.rankmarg.in/admin/questions/equi...,If $R$ is the smallest equivalence relation on...,"Let $R = \{(1,2),(2,3),(3,3)\}$ be a relation ..."


In [ ]:
def confidence(score):
    if score >= 95:
        return "EXACT_DUPLICATE"
    if score >= 88:
        return "STRONG_DUPLICATE"
    return "POSSIBLE_DUPLICATE"

dup_df["confidence"] = dup_df["similarity_score"].apply(confidence)


In [ ]:
from google.colab import files

output_filename = "best_matches_duplicates.csv"
dup_df.to_csv(output_filename, index=False)
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>